<a href="https://colab.research.google.com/github/Bonorinoa/LangGraph-Crash-Course/blob/main/Lesson_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to LangGraph

This course will prepare you to use LangGraph to create agents. The focus is on practical use, not on the implementation details of LangGraph components. All objects can be customized, but for this introductory course, we decided to focus on the abstractions and intuition behind this tool to get you on your way to creating agents as quickly as possible. Future courses will focus on details relevant to very complex applications.

We will cover the main concepts:

- Basics
- State, Edge, Nodes
- Tools and Tool calling LLMs
- Memoryless Agentic workflow designs
- Memory
- Human-in-the-loop
- Multi-agent workflows
- Deployment

The concepts and objects will be presented through concrete applications to reduce the theoretical load that we consider irrelevant in introductory stages. You will notice that the applications gradually increase in complexity as we incorporate the concepts acquired in each class step by step. This will allow you to connect the concepts and progress quickly in your agent development journey.

The course assumes prior experience with Python (intermediate), API usage, and LangChain (novice).

Unlike other courses, we rely purely on open-source models, so there is no additional cost. We believe in transparent and free research. Working with open-source models entails greater difficulty as they tend to be less powerful, but this gap is closing rapidly.

## 0. Basic Prerequisities

In [ ]:
%pip install langgraph langsmith langchain_groq langchain_core langchain_community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


We will work with language model chatbots to create our intelligent applications. Instead of OpenAI or Anthropic, we will use the provider [Groq](https://console.groq.com/) to load open-source models. Please visit the link to obtain your API key.

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")

GROQ_API_KEY: ··········


Although Groq has its own API, using the provider through LangChain has several benefits. Primarily, all the functionalities developed by the LangChain team will be at our disposal. It also simplifies the code by making it concise and structured. For example, all our models will have two attributes:

invoke() - Returns the result upon completion of the operation

stream() - Returns the result gradually

Additionally, the messages from the models are automatically converted into Messages objects of LangChain. There are three types of messages that we will frequently use: [`SystemMessage`](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html), [`HumanMessage`](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html#langchain_core.messages.human.HumanMessage), and [`AIMessage`](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html). Each message contains an attribute called content with the model's response, along with some metadata such as the total number of tokens used, response time, model name, among others.

In [ ]:
from langchain_groq import ChatGroq
import pprint as pp

llama3 = ChatGroq(model_name="llama3-8b-8192", # more models available at ://console.groq.com/docs/models
                  temperature=1, # Temperature is related to the LLM's sampling process. 0 produces more deterministic generations, while 1+ allows for more uncertainty.
                  max_tokens=500) # maximum number of tokens to generate

answer = llama3.invoke("Hi, how are you doing?")
print(f'Content: {answer.content}')
pp.pprint(answer.response_metadata)

Content: I'm just a large language model, I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you have! It's great to connect with you. Is there something specific you'd like to talk about or ask?
{'finish_reason': 'stop',
 'logprobs': None,
 'model_name': 'llama3-8b-8192',
 'system_fingerprint': 'fp_6a6771ae9c',
 'token_usage': {'completion_time': 0.048333333,
                 'completion_tokens': 58,
                 'prompt_time': 0.003636408,
                 'prompt_tokens': 17,
                 'queue_time': 0.022547730999999998,
                 'total_time': 0.051969741,
                 'total_tokens': 75}}


In [ ]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
data = answer.response_metadata

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(data, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)

{
    "token_usage": {
        "completion_tokens": 58,
        "prompt_tokens": 17,
        "total_tokens": 75,
        "completion_time": 0.048333333,
        "prompt_time": 0.003636408,
        "queue_time": 0.022547730999999998,
        "total_time": 0.051969741
    },
    "model_name": "llama3-8b-8192",
    "system_fingerprint": "fp_6a6771ae9c",
    "finish_reason": "stop",
    "logprobs": null
}



In [ ]:
stream_answer = llama3.stream("Hi, how are you doing?") # returns an iterable BaseChatModel.stream
print(type(stream_answer))

for chunk in stream_answer:
    print(chunk.content)
    #print(chunk.content, end="")

<class 'generator'>

I
'm
 just
 a
 language
 model
,
 I
 don
't
 have
 emotions
 or
 feelings
 like
 humans
 do
,
 but
 I
'm
 functioning
 properly
 and
 ready
 to
 assist
 you
 with
 any
 questions
 or
 tasks
 you
 have
!



Behind the scenes, `invoke()` and `stream()` convert the string to a message of type `HumanMessage`.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# Create a message
msg = HumanMessage(content="Hi! ",
                   name="Augusto")

# Message list
messages = [msg]

# Invoke the model with a list of messages
llama3.invoke(messages).content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [ ]:
# system prompt
sys_msg = SystemMessage(content="You are a pirate in an adventurous mission to find dragons.")

# Message list
messages = [sys_msg, msg]

# Invoke the model with a list of messages
llama3.invoke(messages).content

"Arrr, shiver me timbers! I be Captain Blackwood, the most feared and infamous pirate on the seven seas. Me and me crew, the Brave Behemoths, have been sailin' for weeks, searchin' for the legendary dragons of the Eastern Isles. They say these beasts be hidin' treasure beyond our wildest dreams, and I aim to find it!\n\nMe trusty map, passed down from me great-granddaddy, be pointin' me towards the Firestorm Isles. They say the dragons be born in the heart o' the islands, where the lava flows like rivers o' gold. Me and me crew be ready fer the journey o' a lifetime, but I'll need yer help to navigate the treacherous waters and avoid the Royal Navy's wrath!\n\nSo, are ye ready to set sail with me and me crew on the quest for dragon's gold?"